# Online store smart chat assistant
## The group project in artificial intelligence
### Kostiantyn Ostapenko, Olaf Gontarczyk

```css
Pre-installing section
```

In [17]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install tflearn
!{sys.executable} -m pip install tensorflow==1.4

```python
Section of standard packages import
```

In [23]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import tflearn as tf
import tensorflow as tnf
import random
import json
import pickle

```python
Section model generation for the seed of the data
```

In [24]:
with open("intents.json") as file:
    data = json.load(file)
    
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent["tag"] not in labels:
        labels.append(intent["tag"])
        
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    
    wrds = [stemmer.stem(w) for w in doc]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
            
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)
    
training = np.array(training)
output = np.array(output)

tnf.reset_default_graph()

net = tf.input_data(shape=[None, len(training[0])])
net = tf.fully_connected(net, 8)
net = tf.fully_connected(net, 8)
net = tf.fully_connected(net, len(output[0]), activation="softmax")
net = tf.regression(net)

model = tf.DNN(net)

model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 3999  | total loss: 0.05580 | time: 0.003s
| Adam | epoch: 1000 | loss: 0.05580 - acc: 0.9987 -- iter: 24/26
Training Step: 4000  | total loss: 0.05453 | time: 0.004s
| Adam | epoch: 1000 | loss: 0.05453 - acc: 0.9988 -- iter: 26/26
--
INFO:tensorflow:C:\Users\ostap\chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


```python
Prediction method. Going through given phrase compare it with the keywords
```

In [25]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
                
    return np.array(bag)

```python
Communication method. Waiting for the user input. Depend on the model prediction give an answer.
```

In [26]:
def chat():
    print("Start talking with a bot (type quit to stop).")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
            
        results = model.predict([bag_of_words(inp, words)])
        results_index = np.argmax(results)
        tag = labels[results_index]
        
        for tg in data["intents"]:
            if tg["tag"] == tag:
                responses = tg["responses"]
                
        print(random.choice(responses))

1. question exaples:  
    * hello  
    * how are you  
    * could i get something to eat  
    * hours of operation  
    * how old are you  
    * cya  
2. quit for the end conversation  

In [27]:
chat()

Start talking with a bot (type quit to stop).
You: hello
Hi there, how can I help?
You: how old are you
I am 25 years old!
You: could i get something to eat
We sell chocolate chip cookies for $2!
You: hours of operation
We are open 7am-4pm Monday-Friday!
You: how old are you
25 years young!
You: could i get something to eat
We sell chocolate chip cookies for $2!
You: could i get something to eat
Cookies are on the menu!
You: how old are you
25 years young!
You: how old are you
I am 25 years old!
You: cya
Talk to you later
You: quit
